# 1. Adquisición y descripción de los datos

In [ ]:
import pandas as pd
import requests
import re
import warnings

##1.1. Obtencion de datos de Google maps

In [ ]:
with open('API_KEY.txt') as f:
  api_key = f.read()

In [ ]:
# Se crea un arreglo con los nombres de las tiendas de Saga Falabella a nivel nacional obtenidos de la url:
# https://www.falabella.com.pe/falabella-pe/myorders/claimbook
# Nota 1: Se obtiene las tiendas de la página de libro de reclamaciones porque indecopi multa a los comercios que no actualizan la info.
# Nota 2: No se ha obtenido el listado por web scraping dado que la página esta protegida con CloudFlare

stores = ['ANGAMOS', 'AREQUIPA CAYMA', 'AREQUIPA PORONGOCHE', 'ATOCONGO', 'BELLAVISTA',
          'CAJAMARCA', 'CAÑETE', 'CENTRO CIVICO', 'CHICLAYO', 'CHIMBOTE', 'COMAS', 'CUSCO',
          'FALABELLA EXPRESS SJL', 'HUANCAYO', 'HUANUCO', 'HUARAZ EXPRESS', 'ICA MALL', 'IQUITOS',
          'JOCKEY PLAZA', 'LIMA CENTRO', 'LIMA NORTE', 'MALL DEL SUR', 'MEGA PLAZA',
          'MIRAFLORES', 'PIURA CENTRO', 'PIURA MALL', 'PUCALLPA', 'PURUCHUCO', 'SALAVERRY', 'SAN ISIDRO',
          'SAN MIGUEL', 'SANTA ANITA', 'TACNA EXPRESS', 'TRUJILLO MALL']

In [ ]:
def filterStores(stores):
  # Se eliminan del listado las tiendas express dado que tienen público reducido
  list_stores = []
  idx = 0;
  for store in stores:
    if 'EXPRESS' not in store:
      idx += 1
      list_stores.append({'id': str(idx), 'name': store})

  return list_stores

def requestPlaces(urlparams):
  headers = {'Accept': 'application/json'}
  url = 'https://maps.googleapis.com/maps/api/place/' + urlparams + '&language=es&key=' + api_key
  response = requests.get(url, headers=headers)

  data = ''
  if response.status_code == 200:
    data = response.json()
  else:
    print('No se obtuvo resultado de la petición')

  return data

# Aseguramos que el lugar encontrado sea una tienda realmente
def getPlaces(list_stores, prefix=None):
  print('Buscando places en Google Places API...')
  places = []
  for store in list_stores:
    if prefix is not None:
      search = prefix + ' ' + store['name']

    data = requestPlaces('textsearch/json?query=' + search)
    if len(data['results']) > 0:
      types = ['department_store','store']
      filtered_places = [item for item in data['results'] if any(t in item['types'] for t in types)]

      place_data = filtered_places[0]
      place_data['store_id'] = store['id']
      place_data['store_name'] = 'T. ' + store['name']
      places.append(place_data)
    else:
      print('No se obtuvo resultados.')

  return places

def getReviews(place_id):
  reviews = []
  data = requestPlaces('details/json?place_id=$place_id&fields=name,reviews,rating'.replace("$place_id", place_id))

  if 'reviews' in data['result']:
      reviews = data['result']['reviews']
  else:
    print('No se pudo obtener los comentarios de ', place_id)

  return reviews

def getDatasetStoresandReviews(list_places):
  # Filtrar campos de la lista de places
  warnings.filterwarnings('ignore')
  df = pd.DataFrame(columns=['id','store_id','store','address','zip_code','types','rating','total_ratings','location_lat','location_lng'])
  dfr = pd.DataFrame(columns=['store_id','store','author','language','rating','time','text'])
  for idx, place in enumerate(list_places):
    dplace = {'id': place['place_id'],
              'store_id': place['store_id'],
              'store': place['store_name'],
              'address': place['formatted_address'],
              'zip_code': re.search(r'\b\d{5}\b', place['formatted_address']).group(),
              'types': ','.join(place['types']),
              'rating': float(place['rating']),
              'total_ratings': int(place['user_ratings_total']),
              'location_lat': float(place['geometry']['location']['lat']),
              'location_lng': float(place['geometry']['location']['lng'])
           }
    # Obtener reviews desde la API
    reviews = getReviews(place['place_id'])

    if len(reviews) > 0:
      index = 0
      for review in reviews:
        index += 1
        dreview = {'store_id': place['store_id'],
                   'store': place['store_name'],
                   'author': review['author_name'],
                   'language': review['language'],
                   'rating': review['rating'],
                   'time': review['time'],
                   'text': review['text']
        }
        dfr = dfr.append(dreview, ignore_index=True)
    df = df.append(dplace, ignore_index=True)
  return df, dfr

In [ ]:
store_prefix = 'FALABELLA'
filtered_stores = filterStores(stores)
print(filtered_stores)

[{'id': '1', 'name': 'ANGAMOS'}, {'id': '2', 'name': 'AREQUIPA CAYMA'}, {'id': '3', 'name': 'AREQUIPA PORONGOCHE'}, {'id': '4', 'name': 'ATOCONGO'}, {'id': '5', 'name': 'BELLAVISTA'}, {'id': '6', 'name': 'CAJAMARCA'}, {'id': '7', 'name': 'CAÑETE'}, {'id': '8', 'name': 'CENTRO CIVICO'}, {'id': '9', 'name': 'CHICLAYO'}, {'id': '10', 'name': 'CHIMBOTE'}, {'id': '11', 'name': 'COMAS'}, {'id': '12', 'name': 'CUSCO'}, {'id': '13', 'name': 'HUANCAYO'}, {'id': '14', 'name': 'HUANUCO'}, {'id': '15', 'name': 'ICA MALL'}, {'id': '16', 'name': 'IQUITOS'}, {'id': '17', 'name': 'JOCKEY PLAZA'}, {'id': '18', 'name': 'LIMA CENTRO'}, {'id': '19', 'name': 'LIMA NORTE'}, {'id': '20', 'name': 'MALL DEL SUR'}, {'id': '21', 'name': 'MEGA PLAZA'}, {'id': '22', 'name': 'MIRAFLORES'}, {'id': '23', 'name': 'PIURA CENTRO'}, {'id': '24', 'name': 'PIURA MALL'}, {'id': '25', 'name': 'PUCALLPA'}, {'id': '26', 'name': 'PURUCHUCO'}, {'id': '27', 'name': 'SALAVERRY'}, {'id': '28', 'name': 'SAN ISIDRO'}, {'id': '29', 'n

In [ ]:
list_places = getPlaces(filtered_stores, store_prefix)
print('Nro. de tiendas:',len(list_places))

Buscando places en Google Places API...
Nro. de tiendas: 31


In [ ]:
# Dataset stores
df_stores, df_reviews = getDatasetStoresandReviews(list_places)
df_stores.head()

,id,store_id,store,address,zip_code,types,rating,total_ratings,location_lat,location_lng
0,ChIJBe7XKMvJBZERLxXLXN2i0bQ,1,T. ANGAMOS,"Angamos Open Plaza, Av. Angamos Este 1803, Sur...",15038,"department_store,food,point_of_interest,store,...",4.2,3499,-12.112123,-77.011967
1,ChIJCYWX9UJKQpERd-1IXrKzHkA,2,T. AREQUIPA CAYMA,"Mallplaza, Av. Ejército 793, Cayma 04014, Perú",04014,"department_store,food,point_of_interest,store,...",4.0,11065,-16.390443,-71.546741
2,ChIJeZLHYBpLQpERRLRv3_-pNnE,3,T. AREQUIPA PORONGOCHE,"Mall Aventura Plaza, Arequipa, Av. Porongoche ...",04008,"department_store,point_of_interest,store,estab...",3.9,3771,-16.416682,-71.514465
3,ChIJe7R12Gu4BZERgoGU7LEot3Y,4,T. ATOCONGO,"Centro Comercial, Estacionamiento Open Pl. Ato...",15803,"department_store,food,point_of_interest,store,...",4.0,1457,-12.146962,-76.981434
4,ChIJA_TqGVjJBZERWUarkHURiHk,5,T. BELLAVISTA,"Mall Aventura Plaza, Av. Óscar R. Benavides 38...",07011,"department_store,shopping_mall,store,food,poin...",4.1,4285,-12.055882,-77.101058


In [ ]:
# Dataset Reviews
df_reviews.head()

,store_id,store,author,language,rating,time,text
0,1,T. ANGAMOS,Victoria Villacorta,es,4,1696517623,"Este local comercial, a pesar de tener 3 pisos..."
1,1,T. ANGAMOS,Manuel Rojas,es,5,1690573854,Falabella es una tienda por departamentos chil...
2,1,T. ANGAMOS,AGUSTÍN ZAVALETA QUISPE,es,4,1687550534,Buen lugar para hacer las compras para el hoga...
3,1,T. ANGAMOS,Margarita Zevallos,es,2,1674078501,"Estuve hace 2 semanas, y era un caos. En el 1e..."
4,1,T. ANGAMOS,hector garrido,es,5,1692035234,Muy buenas oferta sobre todo horita en tempora...


##1.2. Obtener datos internos

###1.2.1. Datos tiendas

In [ ]:
match_stores = [
    {'id': '1', 'name':'Angamos'},{'id': '2', 'name':'Arequipa Cayma'},{'id': '3', 'name':'Arequipa Porongoche'},{'id': '4', 'name':'Atocongo'},
    {'id': '5', 'name':'Bellavista'},{'id': '6', 'name':'Cajamarca Quinde'},{'id': '7', 'name':'Canete'},{'id': '8', 'name':'Centro Civico'},
    {'id': '9', 'name':'Chiclayo Mall'},{'id': '10', 'name':'Chimbote'},{'id': '11', 'name':'Comas'},{'id': '12', 'name':'Cusco'},
    {'id': '13', 'name':'Huancayo'},{'id': '14', 'name':'Huanuco'},{'id': '15', 'name':'Ica Mall'},{'id': '16', 'name':'Iquitos Mall'},
    {'id': '17', 'name':'Jockey Plaza'},{'id': '18', 'name':'Lima Centro'},{'id': '19', 'name':'Lima Norte'},{'id': '20', 'name':'Mall Del Sur'},
    {'id': '21', 'name':'Megaplaza'},{'id': '22', 'name':'Miraflores'},{'id': '23', 'name':'Piura Centro'},{'id': '24', 'name':'Piura Mall'},
    {'id': '25', 'name':'Pucallpa'},{'id': '26', 'name':'Puruchuco'},{'id': '27', 'name':'Salaverry'},{'id': '28', 'name':'San Isidro'},
    {'id': '29', 'name':'San Miguel'},{'id': '30', 'name':'Santa Anita'},{'id': '31', 'name':'Trujillo Mall'}]

In [ ]:
# Cargar datos de tiendas
df_stores_db = pd.read_csv('retail-stores.csv')
print(df_stores_db.shape)
df_stores_db.head()

(46, 4)


,Zona,Tienda,TC Trx,gasto prom
0,Norte,Trujillo Mall,0.491303,598.530354
1,Lima1,Miraflores,0.426094,425.048147
2,Lima2,Atocongo,0.376608,436.412065
3,Lima2,Centro Civico,0.355720,369.611264
4,Oriente,Pucallpa,0.354915,602.625568


In [ ]:
# Asignar id
for m in match_stores:
  store = [s for s in filtered_stores if s["id"] == m['id']]
  store = store[0]
  df_stores_db.loc[df_stores_db['Tienda'] == m['name'], 'store_id'] = store['id']
  df_stores_db.loc[df_stores_db['store_id'] == store['id'], 'Tienda'] = 'T. ' + store['name']

In [ ]:
# Eliminar tiendas que no estan en la lista
df_stores_db = df_stores_db.dropna(subset=['store_id'])
# Renombrar columnas
df_stores_db = df_stores_db.rename(columns={'Zona':'zone','Tienda':'store','TC Trx':'conv_rate','gasto prom':'avg_spend'})

print(df_stores_db.shape)
df_stores_db.head()

(31, 5)


,zone,store,conv_rate,avg_spend,store_id
0,Norte,T. TRUJILLO MALL,0.491303,598.530354,31
1,Lima1,T. MIRAFLORES,0.426094,425.048147,22
2,Lima2,T. ATOCONGO,0.376608,436.412065,4
3,Lima2,T. CENTRO CIVICO,0.355720,369.611264,8
4,Oriente,T. PUCALLPA,0.354915,602.625568,25


In [ ]:
# No se incluye la tienda IQUITOS: 16 por no tener datos adicionales
df_stores = df_stores[df_stores['store_id'] != '16']
print(df_stores.shape)

# Agregar tasa de conversion
for index, store in df_stores.iterrows():
  row = df_stores_db.loc[df_stores_db['store_id'] == store['store_id']].to_dict('records')[0]
  df_stores.loc[df_stores['store_id'] == row['store_id'], 'conv_rate'] = row['conv_rate']

df_stores.head()

(30, 10)


,id,store_id,store,address,zip_code,types,rating,total_ratings,location_lat,location_lng,conv_rate
0,ChIJBe7XKMvJBZERLxXLXN2i0bQ,1,T. ANGAMOS,"Angamos Open Plaza, Av. Angamos Este 1803, Sur...",15038,"department_store,food,point_of_interest,store,...",4.2,3499,-12.112123,-77.011967,0.325641
1,ChIJCYWX9UJKQpERd-1IXrKzHkA,2,T. AREQUIPA CAYMA,"Mallplaza, Av. Ejército 793, Cayma 04014, Perú",04014,"department_store,food,point_of_interest,store,...",4.0,11065,-16.390443,-71.546741,0.301041
2,ChIJeZLHYBpLQpERRLRv3_-pNnE,3,T. AREQUIPA PORONGOCHE,"Mall Aventura Plaza, Arequipa, Av. Porongoche ...",04008,"department_store,point_of_interest,store,estab...",3.9,3771,-16.416682,-71.514465,0.233449
3,ChIJe7R12Gu4BZERgoGU7LEot3Y,4,T. ATOCONGO,"Centro Comercial, Estacionamiento Open Pl. Ato...",15803,"department_store,food,point_of_interest,store,...",4.0,1457,-12.146962,-76.981434,0.376608
4,ChIJA_TqGVjJBZERWUarkHURiHk,5,T. BELLAVISTA,"Mall Aventura Plaza, Av. Óscar R. Benavides 38...",07011,"department_store,shopping_mall,store,food,poin...",4.1,4285,-12.055882,-77.101058,0.317479


In [ ]:
# Generar dataset final de tiendas
df_stores.to_csv('df_stores.csv', index=False)

###1.2.1. Encuesta tiendas

In [ ]:
# Cargar datos
df_survey = pd.read_excel('retail-survey.xlsx')
df_survey.head()

,tienda_trx,Id,Sub Atributo,Comentario del Encuestado,NPS
0,Megaplaza,98374265,NaN,es fácil de comprar,10
1,Puruchuco,94895502,Ubicación de la Tienda,"Estacionamiento gratis, cerca a mi casa, no ha...",10
2,Jockey Plaza,99161360,Disponibilidad de productos,"Encuentras lo que buscas y caminas segura, y d...",10
3,San Miguel,79926861,Disponibilidad de productos,Por que encuentro todo lo que necesito,10
4,San Isidro,88877134,Calidad del servicio,100% atencion en tiempo con la facilidad de pago,10


In [ ]:
# Verificar datos nulos
df_survey.isnull().sum()

tienda_trx                      0
Id                             22
Sub Atributo                 7598
Comentario del Encuestado     102
NPS                           172
dtype: int64

In [ ]:
# Limpiar datos
# Eliminar columnas innecesarias
df_reviews_s = df_survey.drop(columns=['Id','Sub Atributo','NPS'])
# Eliminar filas con valor nulo
df_reviews_s = df_reviews_s.dropna(axis=0)

df_reviews_s.isnull().sum()

tienda_trx                   0
Comentario del Encuestado    0
dtype: int64

In [ ]:
# Eliminar comentarios tamaño menor de 5
df_reviews_s = df_reviews_s[df_reviews_s['Comentario del Encuestado'].str.len() >= 10]
# Eliminar comentarios con una sola palabra
df_reviews_s = df_reviews_s[df_reviews_s['Comentario del Encuestado'].str.strip().str.contains(r'\s', regex=True, na=False)]
# Eliminar registros de tiendas que no se consideran
df_reviews_s = df_reviews_s[df_reviews_s['tienda_trx'] != 'Crate & Barrel Jockey Plaza']
df_reviews_s.shape

(51183, 2)

In [ ]:
# Tiendas dataset encuesta
df_reviews_s['tienda_trx'].unique()

array(['Megaplaza', 'Puruchuco', 'Jockey Plaza', 'San Miguel',
       'San Isidro', 'Trujillo Mall', 'Miraflores', 'Santa Anita',
       'Centro Civico', 'Piura Mall', 'Huancayo', 'Huanuco', 'Cusco',
       'Lima Norte', 'Arequipa Cayma', 'Mall Del Sur', 'Comas', 'Angamos',
       'Cajamarca Quinde', 'Ica Mall', 'Lima Centro', 'Chiclayo Mall',
       'Chimbote', 'Bellavista', 'Salaverry', 'Canete',
       'Arequipa Porongoche', 'Piura Centro', 'Pucallpa', 'Atocongo'],
      dtype=object)

In [ ]:
# Asignar código a tiendas
for m in match_stores:
  store = [s for s in filtered_stores if s["id"] == m['id']]
  store = store[0]
  df_reviews_s.loc[df_reviews_s['tienda_trx'] == m['name'], 'store_id'] = store['id']
  df_reviews_s.loc[df_reviews_s['store_id'] == store['id'], 'tienda_trx'] = 'T. ' + store['name']

df_reviews_s.head()

,tienda_trx,Comentario del Encuestado,store_id
0,T. MEGA PLAZA,es fácil de comprar,21
1,T. PURUCHUCO,"Estacionamiento gratis, cerca a mi casa, no ha...",26
2,T. JOCKEY PLAZA,"Encuentras lo que buscas y caminas segura, y d...",17
3,T. SAN MIGUEL,Por que encuentro todo lo que necesito,29
4,T. SAN ISIDRO,100% atencion en tiempo con la facilidad de pago,28


In [ ]:
# Renombrar columnas
df_reviews_s = df_reviews_s.rename(columns={'tienda_trx':'store','Comentario del Encuestado':'text'})
df_reviews_s.columns

Index(['store', 'text', 'store_id'], dtype='object')

In [ ]:
# No se incluye la tienda IQUITOS: 16 por no tener datos adicionales
df_reviews = df_reviews[df_reviews['store_id'] != '16']
df_reviews.shape

(150, 7)

In [31]:
# Obtener solo 50 comentarios por tienda

def select_reviews(group, n):
    if len(group) >= n:
        return group.sample(n=n)
    else:
        return group

df_reviews_srnd = df_reviews_s.groupby('store', group_keys=False).apply(select_reviews, 50)
df_reviews_srnd.shape

(1500, 3)

In [32]:
# consolidar comentarios de Google Maps y encuentas
df_reviews_f = pd.concat([df_reviews[['store_id','store','text']], df_reviews_srnd[['store_id','store','text']]], axis=0)
df_reviews_f.head()

,store_id,store,text
0,1,T. ANGAMOS,"Este local comercial, a pesar de tener 3 pisos..."
1,1,T. ANGAMOS,Falabella es una tienda por departamentos chil...
2,1,T. ANGAMOS,Buen lugar para hacer las compras para el hoga...
3,1,T. ANGAMOS,"Estuve hace 2 semanas, y era un caos. En el 1e..."
4,1,T. ANGAMOS,Muy buenas oferta sobre todo horita en tempora...


In [33]:
# Generar dataset final de reseñas
df_reviews.to_csv('df_reviews_maps.csv', index=False)
df_reviews_f.to_csv('df_reviews.csv', index=False)